In [1]:
!pip install transformers datasets faiss-cpu langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling lan

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from datasets import load_dataset
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain import LLMChain

# Step 1: Load the Full Train Split in Streaming Mode
dataset = load_dataset("trivia_qa", "rc", split="train", streaming=True)

# Step 2: Initialize Smaller Model Components
# Using distilbert-base-uncased-distilled-squad as a smaller question-answering model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

# Set up the pipeline for question-answering
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Define a function to generate a response using the question-answering pipeline
def generate_response(query):
    # Process only the first 100 entries from the streaming dataset
    for i, entry in enumerate(dataset):
        if i >= 100:  # Limit to 100 entries
            break
        context = entry["question"]  # TriviaQA questions are used here as context for simplicity
        result = qa_pipeline({
            "question": query,
            "context": context
        })
        print("Context:", context)
        print("Answer:", result['answer'])
        return result['answer']  # Return only the first answer for simplicity

# Test the pipeline
if __name__ == "__main__":
    query = "Explain retrieval-augmented generation."
    print("Response:", generate_response(query))


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Context: Which American-born Sinclair won the Nobel Prize for Literature in 1930?
Answer: Which American-born Sinclair
Response: Which American-born Sinclair


In [5]:
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering, pipeline
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain

# Step 1: Load the dataset
dataset = load_dataset("trivia_qa", "rc", split="train", streaming=True)

# Step 2: Initialize Embedding and Question-Answering Models
# Smaller model for embeddings
embed_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
embed_model = AutoModel.from_pretrained("distilbert-base-uncased")

# Question-answering model
qa_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)

# Step 3: Embed Contexts for Retrieval
def embed_text(text):
    inputs = embed_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = embed_model(**inputs)
    # Use mean pooling to obtain a fixed-size embedding
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

# Embed the first 100 contexts from the dataset
contexts = []
embeddings = []
for i, entry in enumerate(dataset):
    if i >= 100:
        break
    context = entry["question"]
    contexts.append(context)
    embeddings.append(embed_text(context))

# Convert embeddings to a single numpy array for similarity comparisons
embeddings = np.vstack(embeddings)

# Step 4: Define a Function for Retrieving the Most Similar Context
def retrieve_context(query, contexts, embeddings, top_k=3):
    query_embedding = embed_text(query)
    # Calculate cosine similarity with all contexts
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    # Get indices of top k most similar contexts
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    # Retrieve the top k contexts
    return [contexts[idx] for idx in top_indices]

# Step 5: Set Up LangChain Prompt Template
prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template="Question: {query}\nContext: {context}\nAnswer:"
)
llm_chain = LLMChain(prompt=prompt_template, llm=HuggingFacePipeline(pipeline=qa_pipeline))

# Step 6: Generate Response with Iterative Retrieval-Augmented Generation
def generate_response(query):
    # Retrieve top 3 contexts for the query
    top_contexts = retrieve_context(query, contexts, embeddings)

    # Initialize an empty answer
    final_answer = ""

    # Iteratively use retrieved contexts to improve answer
    for context in top_contexts:
        result = qa_pipeline({
            "question": query,
            "context": context
        })
        answer = result["answer"]
        print("Context:", context)
        print("Answer:", answer)

        # Concatenate answers for demonstration
        final_answer += answer + " "

    return final_answer.strip()

# Test the pipeline
if __name__ == "__main__":
    query = "Explain retrieval-augmented generation."
    print("Final Response:", generate_response(query))


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Context: Banting and Best pioneered the use of what?
Answer: Banting and Best pioneered the use of what
Context: In what decade were video recorders first developed?
Answer: In what decade
Context: In which country did he widespread use of ISDN begin in 1988?
Answer: 1988
Final Response: Banting and Best pioneered the use of what In what decade 1988
